In [22]:
# Pythonの基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# YoutubeAPIの利用
from apiclient.discovery import build
from apiclient.errors import HttpError

# ファイル操作
import os
import glob

# Jupyter上にHTMLを表示する
from IPython.display import HTML

In [23]:
# Youtube APIを呼び出す

# APIキーをファイルから取得
f = open('secret/apikey', 'r')
api_key = f.read()
f.close()

# APIキーを用いてリクエスト用のクラスを作成
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=api_key)

In [24]:
# 事前準備

# 1チャンネルから取得する動画数(最大50)
maxResults = 50

# 参照するチャンネル一覧
# filename = "input/hololive_channels.csv"
filename = "input/okakoro_channels.csv"
df = pd.read_csv(filename, header=0)

# 保存するディレクトリを作成
os.makedirs('output/okakoro', exist_ok=True)

# 途中から取得する場合は下記を変更する
id_start = 0
id_end = df.shape[0]
channel_ids = np.array(df['id'][id_start:id_end])
channel_names = np.array(df['チャンネル'][id_start:id_end])

In [25]:
# ISO表記の動画時間を秒に変換
def pt2sec(pt_time):
    s_list, m_list, h_list = [], [], []
    conc_s, conc_m, conc_h = '', '', ''
    flag = ''
    
    for i in reversed(pt_time):
        if i == 'S':
            flag = 'S'
        elif i == 'M':
            flag = 'M'
        elif i == 'H':
            flag = 'H'
        elif i == 'T':
            break
        else:
            if flag == 'S':
                s_list.append(i)
            elif flag == 'M':
                m_list.append(i)
            elif flag == 'H':
                h_list.append(i)
    
    for s in reversed(s_list):
        conc_s += s
    for m in reversed(m_list):
        conc_m += m
    for h in reversed(h_list):
        conc_h += h
    conc_s = 0 if conc_s == '' else int(conc_s)
    conc_m = 0 if conc_m == '' else int(conc_m)
    conc_h = 0 if conc_h == '' else int(conc_h)

    times = conc_h*3600 + conc_m*60 + conc_s
    return times

# 変換が正しく行われているかテスト
def test_pt2sec():
    if 2*3600 + 30*60 + 30 != pt2sec('PT2H30M30S'):
        print('Test1 Failed')
        return False
    if 30*60 + 30 != pt2sec('PT30M30S'):
        print('Test2 Failed')
        return False
    if 2*3600 + 30*60 != pt2sec('PT2H30M'):
        print('Test3 Failed')
        return False
    if 4*60 != pt2sec('PT4M'):
        print('Test4 Failed')
        return False
    if 0 != pt2sec('PT0S'):
        print('Test5 Failed')
        return False
    return True

print(test_pt2sec())

True


In [26]:
# 動画情報を取得する

for i, channel_id in enumerate(channel_ids):
    channel_name = channel_names[i]
    print(channel_name, ":", i+1, "/", channel_ids.size)
    
    # 各チャンネルから全ての動画IDを取得する
    video_ids = []
    nextPageToken = 'search_start'
    while True:
        if nextPageToken == 'search_start':            
            try:
                videos = youtube.search().list(
                    part = 'id', 
                    channelId = channel_id, 
                    order = 'viewCount',
                    type = "video",
                    maxResults = maxResults, 
                ).execute()
            # HTTPエラー（主にQuota上限）だった場合
            except HttpError:
                print('データ参照中にエラーが発生しました')
                break
        else:
            try:
                videos = youtube.search().list(
                    part = 'id', 
                    channelId = channel_id, 
                    order = 'viewCount',
                    type = 'video',
                    pageToken = nextPageToken,
                    maxResults = maxResults, 
                ).execute()      
            # HTTPエラー（主にQuota上限）だった場合
            except HttpError:
                print('データ参照中にエラーが発生しました')
                break

        # 動画のIDを取得する
        for video_item in videos['items']:
            video_ids.append(video_item['id']['videoId'])
        
        # 次のページが存在しない場合終了します
        if 'nextPageToken' in videos.keys():
            nextPageToken = videos['nextPageToken']
        else:
            break
    
    # 動画のIDから詳細情報を取得する
    video_details = []
    for video_id in video_ids:
        try:
            video_detail = youtube.videos().list(
                part = 'snippet,statistics,contentDetails', 
                id = video_id, 
            ).execute()
        except HttpError:
            print('データ参照中にエラーが発生しました')
            break
            
        video_snippet = video_detail['items'][0]['snippet']
        video_statistics = video_detail['items'][0]['statistics']
        video_contentDetails = video_detail['items'][0]['contentDetails']
        # snippetから取得
        date = video_snippet['publishedAt']
        title = video_snippet['title']
        thumbnail = video_snippet['thumbnails']['high']['url']
        description = video_snippet['description']
        category_id = video_snippet['categoryId']
        # contentDetailsから取得        
        duration = pt2sec(video_contentDetails['duration'])
        duration_origin = video_contentDetails['duration']
        # statisticsから取得
        views = video_statistics['viewCount']
        # 評価数、コメントが非公開の場合は0で埋める
        like = 0
        dislike = 0
        comments = 0
        if 'likeCount' in video_statistics.keys():
            like = video_statistics['likeCount']
        if 'dislikeCount' in video_statistics.keys():
            dislike = video_statistics['dislikeCount']
        if 'commentCount' in video_statistics.keys():
            comments = video_statistics['commentCount']
        # リストのリストとして情報を格納する
        video_details.append([video_id, channel_name, date, title, thumbnail, category_id, duration, duration_origin, description, views, like, dislike, comments])
    
    # video_idが取得できていない場合スキップする
    if len(video_ids) != 0:
        video_details_numpy = np.array(video_details)
        video_details_pandas = pd.DataFrame(data=video_details_numpy, 
                                     columns=['Id', 'Channel', 'Date', 'Title', 'Thumbnail', 'CategoryId', 'Duration', 'DurationOriginal', 'Description', 'Viewcount', 'LikeCount', 'DislikeCount', 'CommentCount'])
        # データが大きいためチャンネルごとに保存する
        video_details_pandas.to_csv('output/okakoro/'+channel_name+'_videos.csv')

print('データ取得が完了 or データ取得を途中で終了しました')

猫又おかゆ : 1 / 2
戌神ころね : 2 / 2
データ取得が完了 or データ取得を途中で終了しました


In [27]:
# 取得した情報を確認する

holo_filenames = glob.glob("output/okakoro/*")
for i, holo_filename in enumerate(holo_filenames):
    holo_videos = pd.read_csv(holo_filename, index_col=0)
    # テーブルを結合する
    if i == 0:
        all_holo_videos = holo_videos
    else:
        all_holo_videos = pd.concat([all_holo_videos, holo_videos])

# 再生数でソート
sorted_holo_videos = all_holo_videos.sort_values(by='Viewcount', ascending=False)

# Table形式で表示
top_n = 10
pd.set_option('display.max_rows', top_n)
# sorted_holo_videos.head(top_n)

# HTMLに変換し表示
html = '<h1>再生数ソート</h1>'
html += '<div style="float:left;">'
for i in range(top_n):
    html += ('<img src="'+np.array(sorted_holo_videos['Thumbnail'])[i] +' "alt="取得できませんでした" width="100">')
    # サンプルIDを入れておく
    html += ('<a href="https://www.youtube.com/watch?v='+np.array(sorted_holo_videos['Id'])[i]+'">#'+str(i+1)+'. '+np.array(sorted_holo_videos['Title'])[i]+'</a><br>')
html += '</div>'

HTML(html)